# Generate expected results for back tests

They save results to `.csv` and use these to verify that the model produces consistent results.

In [1]:
# pylint: disable=missing-module-docstring
%load_ext autoreload
%autoreload 1
%aimport simulation

# pylint: disable=wrong-import-position
import os
import time

from IPython.display import display
import pandas as pd

from simulation import Model, Param, Runner

In [2]:
# Start timer
start_time = time.time()

In [3]:
# Define path to folder for expected results for tests
TESTS = "../tests/exp_results/"

The raw `audit_list` from a single run of the model (converted to a dataframe).

In [4]:
# Run model
model = Model(param=Param(), run_number=0)
model.run()

# Convert audit list to dataframe and preview
audit_list = pd.DataFrame(model.audit_list)
display(audit_list.head())

# Save to csv
audit_list.to_csv(os.path.join(TESTS, "audit_list.csv"), index=False)

,time,asu_occupancy,rehab_occupancy
0,1095,8,13
1,1096,7,14
2,1097,8,14
3,1098,8,14
4,1099,8,14


The dataframes returned by `run_single()` from `Runner`, from a single replication.

In [5]:
# Run model
runner = Runner(param=Param())
runner_df = runner.run_single(run=0)

# Preview the dataframes
display(runner_df["asu"].head())
display(runner_df["rehab"].head())

# Save to csv
runner_df["asu"].to_csv(os.path.join(TESTS, "asu_occupancy.csv"), index=False)
runner_df["rehab"].to_csv(os.path.join(TESTS, "rehab_occupancy.csv"),
                          index=False)

,beds,freq,pct,c_pct,prob_delay,1_in_n_delay
0,1,2,0.001096,0.001096,1.000000,1.0
1,2,12,0.006575,0.007671,0.857143,1.0
2,3,57,0.031233,0.038904,0.802817,1.0
3,4,104,0.056986,0.095890,0.594286,2.0
4,5,162,0.088767,0.184658,0.480712,2.0


,beds,freq,pct,c_pct,prob_delay,1_in_n_delay
0,2,1,0.000548,0.000548,1.000000,1.0
1,3,8,0.004384,0.004932,0.888889,1.0
2,4,35,0.019178,0.024110,0.795455,1.0
3,5,47,0.025753,0.049863,0.516484,2.0
4,6,129,0.070685,0.120548,0.586364,2.0


## Run time

In [6]:
# Get run time in seconds
end_time = time.time()
runtime = round(end_time - start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 0s
